### CS5228 Mini Project Team 07 - Data Preprocessing

#### Objective

This notebook aims to preprocess the raw dataset to:
- Handle missing values and perform data cleaning
- Encode categorical features
- Normalize numerical features

The output of this notebook create a pickle file `cleaned_datasets.pkl` containing variants of train and test datasets. Structure:
```python
{
    "standard_scale": 
    {
        "train": train_df_with_standard_scaling,
        "test": test_df_with_standard_scaling
    },

    "min_max_scale": 
    {
        "train": train_df_with_min_max_scaling,
        "test": test_df_with_min_max_scaling
    } 
}
```

#### Imports

In [187]:
import pandas as pd
import pickle
from sklearn.preprocessing import MinMaxScaler, StandardScaler

#### Read the raw dataset

In [188]:
train_df = pd.read_csv('./dataset/churn-bigml-80.csv')
test_df = pd.read_csv('./dataset/churn-bigml-20.csv')

In [189]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2666 entries, 0 to 2665
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   State                   2666 non-null   object 
 1   Account length          2666 non-null   int64  
 2   Area code               2666 non-null   int64  
 3   International plan      2666 non-null   object 
 4   Voice mail plan         2666 non-null   object 
 5   Number vmail messages   2666 non-null   int64  
 6   Total day minutes       2666 non-null   float64
 7   Total day calls         2666 non-null   int64  
 8   Total day charge        2666 non-null   float64
 9   Total eve minutes       2666 non-null   float64
 10  Total eve calls         2666 non-null   int64  
 11  Total eve charge        2666 non-null   float64
 12  Total night minutes     2666 non-null   float64
 13  Total night calls       2666 non-null   int64  
 14  Total night charge      2666 non-null   

In [190]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667 entries, 0 to 666
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   State                   667 non-null    object 
 1   Account length          667 non-null    int64  
 2   Area code               667 non-null    int64  
 3   International plan      667 non-null    object 
 4   Voice mail plan         667 non-null    object 
 5   Number vmail messages   667 non-null    int64  
 6   Total day minutes       667 non-null    float64
 7   Total day calls         667 non-null    int64  
 8   Total day charge        667 non-null    float64
 9   Total eve minutes       667 non-null    float64
 10  Total eve calls         667 non-null    int64  
 11  Total eve charge        667 non-null    float64
 12  Total night minutes     667 non-null    float64
 13  Total night calls       667 non-null    int64  
 14  Total night charge      667 non-null    fl

Based on the overview above, there is no missing entry in both train and test datasets. Now we split numerical and categorical features for further EDA and data preprocessing.

Numerical columns: 'Account length ', 'Number vmail messages', 'Total day minutes', 'Total day calls', 'Total day charge', 'Total eve minutes', 'Total eve calls', 'Total eve charge', 'Total night minutes', 'Total night calls', 'Total night charge', 'Total intl minutes', 'Total intl calls', 'Total intl charge', 'Customer service calls'

Categorical columns: 'State', 'Area code', 'International plan', 'Voice mail plan', 'Churn'(which is the target column)

In [191]:
train_df.columns

Index(['State', 'Account length', 'Area code', 'International plan',
       'Voice mail plan', 'Number vmail messages', 'Total day minutes',
       'Total day calls', 'Total day charge', 'Total eve minutes',
       'Total eve calls', 'Total eve charge', 'Total night minutes',
       'Total night calls', 'Total night charge', 'Total intl minutes',
       'Total intl calls', 'Total intl charge', 'Customer service calls',
       'Churn'],
      dtype='object')

In [192]:
numerical_columns = ['Account length', 'Number vmail messages', 'Total day minutes', 'Total day calls', 'Total day charge', 'Total eve minutes', 'Total eve calls', 'Total eve charge', 'Total night minutes', 'Total night calls', 'Total night charge', 'Total intl minutes', 'Total intl calls', 'Total intl charge', 'Customer service calls']
categorical_columns = ['State', 'Area code', 'International plan', 'Voice mail plan', 'Churn']

#### Categorical feature preprocessing

Observation: 

For the Area code feature, we can perform OHE given it only has 3 unique values and they appear in both train and test sets.

Since the State feature contains 51 unique values, applying One-Hot Encoding (OHE) would result in a large number of new features. Given that our dataset has only 2,666 entries, this could lead to the curse of dimensionality and potential overfitting. To mitigate this, two alternative encoding methods are considered:

- Label Encoding: Assigns a unique numeric value to each state. However, this method may introduce an unintended ordinal relationship among categorical values.
- Frequency Encoding: Replaces each state with its relative occurrence in the dataset, capturing its distribution without implying any order.

To avoid data ;eakage in test dataset:
- Label Encoding: The mapping derived from the training set should be applied to the test set, and any unseen categories should be assigned a default value 0.
- Frequency Encoding: The frequency mapping from the training set should be used for the test set, and any unseen states should be assigned the mean frequency from the training distribution.

In [193]:
train_df['Area code'].unique(), train_df['State'].unique().size

(array([415, 408, 510], dtype=int64), 51)

In [194]:
test_df['Area code'].unique(), test_df['State'].unique().size

(array([408, 415, 510], dtype=int64), 51)

Functions to encode categorical features

In [195]:
def perform_binary_encoding(train_df, test_df, columns):
    for column in columns:
        train_df[column] = train_df[column].map({'Yes': 1, 'No': 0})
        test_df[column] = test_df[column].map({'Yes': 1, 'No': 0})
    return train_df, test_df

def perform_one_hot_encoding(train_df, test_df, columns):
    train_df = pd.get_dummies(train_df, columns=columns)
    test_df = pd.get_dummies(test_df, columns=columns)
    return train_df, test_df

def perform_frequency_encoding(train_df, test_df, columns):
    for column in columns:
        new_col_name = column + '_freq'
        freq_encoding = train_df[column].value_counts(normalize=True)
        train_df[new_col_name] = train_df[column].map(freq_encoding)
        test_df[new_col_name] = test_df[column].map(freq_encoding)
        test_df[new_col_name].fillna(freq_encoding.mean(), inplace=True)
    return train_df, test_df

def perform_label_encoding(train_df, test_df, columns):
    for column in columns:
        new_col_name = column + '_label'
        label_encoding = {k: i for i, k in enumerate(train_df[column].unique(), 0)}
        train_df[new_col_name] = train_df[column].map(label_encoding)
        test_df[new_col_name] = test_df[column].map(label_encoding)
    return train_df, test_df

def drop_columns(train_df, test_df, columns):
    return train_df.drop(columns=columns, axis=1), test_df.drop(columns=columns, axis=1)

def rearrage_columns(train_df, test_df, columns, target):
    for column in columns:
        freq_col = column + '_freq'
        label_col = column + '_label'
        ohe_col = 'ohe_' + column
        cols = train_df.columns.tolist()
        raw_idx = cols.index(column)
        cols.insert(raw_idx + 1, cols.pop(cols.index(freq_col)))
        cols.insert(raw_idx + 2, cols.pop(cols.index(label_col)))
        train_df = train_df[cols]
        test_df = test_df[cols]
    
    # Move target column to the end
    cols = train_df.columns.tolist()
    cols.insert(len(cols), cols.pop(cols.index(target)))
    train_df = train_df[cols]
    test_df = test_df[cols]
    
    return train_df, test_df

In [196]:
train_df, test_df = perform_binary_encoding(train_df, test_df, ['International plan', 'Voice mail plan'])
train_df, test_df = perform_one_hot_encoding(train_df, test_df, ['Area code'])
train_df, test_df = perform_frequency_encoding(train_df, test_df, ['State'])
train_df, test_df = perform_label_encoding(train_df, test_df, ['State'])
train_df, test_df = rearrage_columns(train_df, test_df, ['State'], 'Churn')
train_df, test_df = drop_columns(train_df, test_df, ['State'])


#### Numerical feature preprocessing

To avoid data leakage, we fit the scaler only on the training set and use the fit scaler to transform both train and test sets.

Function to normalize numerical features

In [197]:
def perform_scaling(train_df, test_df, columns, scaler):
    train_df[columns] = scaler.fit_transform(train_df[columns])
    test_df[columns] = scaler.transform(test_df[columns])
    return train_df, test_df

In [198]:
min_max_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

train_df_standard, test_df_standard = perform_scaling(train_df, test_df, numerical_columns, min_max_scaler)
train_df_min_max, test_df_min_max = perform_scaling(train_df, test_df, numerical_columns, standard_scaler)

Save the preprocessed df as a pickle file

In [199]:
data_dict = {
    "standard_scale": 
    {
        "train": train_df_standard,
        "test": test_df_standard
    },

    "min_max_scale": 
    {
        "train": train_df_min_max,
        "test": test_df_min_max
    } 
}

with open('./dataset/cleaned_datasets.pkl', 'wb') as f:
    pickle.dump(data_dict, f)